In [1]:
import re
import scipy
from scipy import linalg
import scipy.optimize
import numpy
from numpy import array, log, dot, zeros, exp
from numpy.random import rand

In [ ]:
#parse reactions
class Reaction:
    def __init__(self,reaction_string,K):
        self.K=K
        
        def extract_reagetns(s):
            if s == "":
                return {}
            s_reagents=s.split(" + ")
            res={}

            for r in s_reagents:
                r=r.strip()
                if r.find(" ")>0:
                    [n,r]=r.split(" ")
                    res[r]=int(n)
                else:
                    res[r]=1

            return res

        [self.reagents,self.products] = map(extract_reagetns,reaction_string.split("<=>"))
        
r=Reaction('Fe3+ + 2 e- <=> Fe2+', 4.4e+4)
assert({"Fe3+":1, "e-":2}==r.reagents)
assert({"Fe2+":1} == r.products)
assert(4.4e+4 == r.K)

def squarify(M):
    """makes a square matrix from M by padding"""
    sz=M.shape
    M_sq=zeros((max(sz),max(sz)))
    M_sq[:sz[0],:sz[1]]=M
    return M_sq

def null(A, eps=1e-15):
    """calculates null space of the matrix A"""
    u, s, vh = scipy.linalg.svd(A)
    null_mask = (s <= eps)
    null_space = scipy.compress(null_mask, vh, axis=0)
    return scipy.transpose(null_space)

def null_sq(A, eps=1e-15):
    """Like null but squarifies A before calculation"""
    u, s, vh = scipy.linalg.svd(squarify(A))
    null_mask = (s <= eps)
    null_space = scipy.compress(null_mask, vh, axis=0)
    return scipy.transpose(null_space)

#make a list of equations to use in 
#null space solver
def calculate_rates(all_components,reactions):
    """Represents reaction molality of reagents and products in matrix form. 
    Each reaction is represented by a row, where the values are proportional to the 
    number of moles of reagents and products.
    
    Assume we have a reaction
    
    A + 2B <=> 3C 
    
    and also that the order of reagents is A,B,C, then this reaction will be respesented as a vector
    
    +1  +2  -3"""
    init_coefficients={}

    for component in all_components:
        init_coefficients[component]=0

    coefficient_matrix=[]
    reaction_constants=[]
        
    for reaction in reactions:
        coefficients=init_coefficients.copy()
        
        for reagent,n in reaction.reagents.iteritems():
            coefficients[reagent]-=n
            
        for product,n in reaction.products.iteritems():
            coefficients[product]+=n
            
        coefficient_matrix.append(coefficients.values())
        reaction_constants.append(reaction.K)

    return array(coefficient_matrix),log(array(map(float,reaction_constants))),coefficients.keys()

def equilibrate_solution(ph,reactions,initial_concentrations):
    """Main equilibration funciton. Given solution ph, list of reactions and initial concentrations
    returns the concentration of the products after solution is fully equilibrated"""
    
    reactions=reactions[:]
    
    if not ph is None:
        h=10**-ph
        #Ph is presented by adding the infinite sources of water, H+ and OH- ions
        reactions.append(Reaction('<=>H+',  h))
        reactions.append(Reaction('<=>OH-', (1e-14/h)))
        reactions.append(Reaction('<=>H2O', (1-h-1e-14/h)))
    else:
        #Else ph is determined by water dissociation reaction and all the other reactions
        reactions.append(Reaction('H2O<=>OH- + H+', 1e-14),)
    
    #List all possible ions
    all_components = set()
    for r in reactions:
        all_components=all_components.union(set(r.reagents.keys()))
        all_components=all_components.union(set(r.products.keys()))

    #Initialize concentrations to 0
    initial = {}
    for c in all_components:
        initial[c]=0
        
    #Update the ones from the input
    initial.update(initial_concentrations)

    A,b,species=calculate_rates(all_components,reactions)
    
    #At this moment, the problem got reduced to almost linear algebra.
    #We need to find a vector x of logarithms of final concentrations such that
    # A*x=b
    #Where A is the matrix of reagent/product molality, x is the vector of logarithms of 
    #final product concentrations and b is a vector of logarithms of reaction constants
    #
    #This linear system is underdetermined since there is less reactions than ions 
    #(for example there will never be a reaction turning H+ into O2-)
    #the rest of the variables are determined by initial concentrations
    #null(A)*exp(x)=null(A)*i
    #where null(A) is the null space of the matrix A and i is a vector of initial concentrations

    #solution thus consists of two parts. The solution of the first equation is calculated by pseudoinverse of A
    x=dot(linalg.pinv(A),b)
    
    #the other variables are calculated in the following:
    N=null_sq(A)
    spec_to_n={}
    for s,n in enumerate(species):
        spec_to_n[s]=n
    initial_vector=[]
    for s in species:
        initial_vector.append(initial[s])

    initial_vector=array(initial_vector) 
    invariants=dot(N.T,initial_vector)
    sz_i=len(invariants)

    def current_invariants(ksi):
        return dot(N.T,exp(x+dot(N,ksi)))

    quality = current_invariants(zeros(sz_i))

    #Currently solver fails sometimes. This is due to the fact that even those reactions which cannot possibly happen
    #are still present in the solution matrix. This makes convergence non-guaranteed. 
    #To find solution I simply restart equilibration with random seeds until it converges.
    while(True):
        to_optimize=lambda ksi:current_invariants(ksi)-invariants
        ksi=scipy.optimize.leastsq(to_optimize,1e-5*rand(sz_i))[0]
        if all(to_optimize(ksi)<1e-15):
            break

    resulting_concentrations=exp(x+dot(N,ksi))

    result={}
    for n,s in enumerate(species):
        result[s]=resulting_concentrations[n]
        
    return result

def print_concentrations(concentrations):
    for s in concentrations:
        print "%15s %.1e" % (s,concentrations[s])

In [10]:
#User interface to the above calculations. This is the part I would like to be accessible for the users from 
#a web-page

#Make a list of chemical reactions and their constants

reactions = [
Reaction('Fe(OH)3(s) + 3 H+ <=> Fe3+ + 3 H2O', 10**3.54),
Reaction('Fe4(Fe(CN)6)3 <=> 4 Fe3+ + 3 Fe(CN)6_3- + 3 e-', 10**-84.5),
Reaction('Fe3+ + e- <=> Fe2+', 10**13),
Reaction('Fe3+ + OH- + e-<=>Fe(OH)+', 10**6.3),
Reaction('Fe3+ + OH- <=> Fe(OH)2+', 10**-2.2),
Reaction('Fe3+ + 2 OH- <=> Fe(OH)2_+', 10**-5.7),
Reaction('Fe3+ + 2 OH- + e- <=> Fe(OH)2_0', 10**-3),
Reaction('Fe3+ + 3 OH- + e- <=> Fe(OH)3_-', 10**-19),
Reaction('Fe3+ + 3 OH- <=> Fe(OH)3_0', 10**-13.2),
Reaction('Fe3+ + 4 OH- <=> Fe(OH)4_-', 10**-21.6),

Reaction('Fe(CN)6_3- + e- <=> Fe(CN)6_4-', 10**-6),
Reaction('Fe(CN)6_3- + e- + H+ <=> HFe(CN)6_3-', 10**10.4),
Reaction('Fe(CN)6_3- + e- + 2 H+ <=> H2Fe(CN)6_2-', 10**12.8),
Reaction('Fe(CN)6_3- + e- + K+ <=> KFe(CN)6_3-', 10**8.5),
Reaction('Fe(CN)6_3- + K+ <=> KFe(CN)6_2-', 10**1.5),
Reaction('Fe(CN)6_3- + e- + Ca2+ <=> CaFe(CN)6_2-', 10**10.1),
Reaction('Fe(CN)6_3- + Ca2+ <=> CaFe(CN)6_-', 10**2.6),
    
Reaction('HEDTA3- + H+ <=> H-HEDTA2-',10**10.5),
Reaction('HEDTA3- + 2 H+ <=> H2HEDTA1-',10**16.3),
Reaction('HEDTA3- + 3 H+ <=> H3HEDTA0',10**19.1),
Reaction('HEDTA3- + Fe3+ <=> FeHEDTA0',10**21.7),
Reaction('HEDTA3- + Fe3+ + e-<=> FeHEDTA-',10**26.5),
Reaction('HEDTA3- + Fe3+ + e- + H+<=> FeHHEDTA0',10**29.5),
Reaction('HEDTA3- + Fe3+ + e- + OH-<=> Fe(OH)HEDTA2-',10**17.1),
Reaction('HEDTA3- + Fe3+ + OH-<=> Fe(OH)HEDTA-',10**17.6),
Reaction('HEDTA3- + Fe3+ + 2 OH-<=> Fe(OH)2HEDTA2-',10**8.2),
Reaction('HEDTA3- + Ca2+<=> CaHEDTA-',10**9.5)
]

#Set solution ph and initial ion conentrations
ph=7.0
initial={"Fe4(Fe(CN)6)3":0.1, "Ca2+":0.00, "Cit":0.0, "HEDTA3-":0.5}

solution = equilibrate_solution(7.0,reactions,initial)
print_concentrations(solution)

       FeHEDTA- 2.6e-01
      H2HEDTA1- 1.0e-02
        Fe(OH)+ 3.2e-24
             K+ 2.4e-55
   Fe(OH)HEDTA- 5.3e-13
     Fe(OH)3(s) 7.4e-02
  Fe(OH)HEDTA2- 1.0e-17
       FeHEDTA0 6.6e-02
      Fe(OH)3_- 1.6e-63
    CaFe(CN)6_- 1.1e-14
    KFe(CN)6_2- 1.9e-54
           Ca2+ 1.0e-16
       Fe(OH)2+ 1.6e-28
     Fe(CN)6_3- 2.6e-01
           Fe3+ 2.6e-19
   CaFe(CN)6_2- 2.1e-11
  Fe4(Fe(CN)6)3 5.6e-05
      Fe(OH)3_0 1.6e-53
             e- 6.2e-05
            OH- 1.0e-07
      Fe(OH)2_0 1.6e-40
        HEDTA3- 5.2e-05
   H2Fe(CN)6_2- 1.0e-06
 Fe(OH)2HEDTA2- 2.1e-29
    HFe(CN)6_3- 4.0e-02
      FeHHEDTA0 2.6e-05
      Fe(OH)2_+ 5.1e-39
      H-HEDTA2- 1.6e-01
      Fe(OH)4_- 6.4e-69
    KFe(CN)6_3- 1.2e-51
           Fe2+ 1.6e-10
       CaHEDTA- 1.7e-11
             H+ 1.0e-07
            H2O 1.0e+00
       H3HEDTA0 6.5e-07
     Fe(CN)6_4- 1.6e-11
